<a href="https://colab.research.google.com/github/KillerX629/KillerX629/blob/main/vicuna_index_GPTQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vicuna index

## Implementacion en google collab.

Usamos un modelo de lenguaje capaz de completar instrucciones para responder, de forma informada y sin permitir "alucinaciones" del modelo de IA, preguntas sobre un material determinado


In [ ]:
import os
os.environ['TORCH_CUDA_ARCH_LIST'] = '6.0;6.1;7.0;7.5'


!git clone https://github.com/qwopqwop200/GPTQ-for-LLaMa -b cuda
%cd GPTQ-for-LLaMa
!pip install -r requirements.txt
!python setup_cuda.py install
%cd ..

fatal: destination path 'GPTQ-for-LLaMa' already exists and is not an empty directory.
/content/GPTQ-for-LLaMa
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-_7mf4sdf
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-_7mf4sdf
  Resolved https://github.com/huggingface/transformers to commit 6397b7f00804b1ebdd1ed3612fc1abdd8cc07659
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.30.0.dev0-py3-none-any.whl size=7120361 sha256=343a76bf1a1e57ba4dddea7e54675d22f1c2bab6b8aa6cf0e6b0be62d5fc921d
  Stored in directory: /tmp/pip-ephem-wheel-cache-yrz5qf8r/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempt

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp
%cd llama.cpp
!make LLAMA_CUBLAS=1
%cd ..
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

fatal: destination path 'llama.cpp' already exists and is not an empty directory.
/content/llama.cpp
I llama.cpp build info: 
I UNAME_S:  Linux
I UNAME_P:  x86_64
I UNAME_M:  x86_64
I CFLAGS:   -I.              -O3 -std=c11   -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wdouble-promotion -Wshadow -Wstrict-prototypes -Wpointer-arith -pthread -march=native -mtune=native -DGGML_USE_CUBLAS -I/usr/local/cuda/include -I/opt/cuda/include -I/targets/x86_64-linux/include
I CXXFLAGS: -I. -I./examples -O3 -std=c++11 -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wno-multichar -pthread -march=native -mtune=native -DGGML_USE_CUBLAS -I/usr/local/cuda/include -I/opt/cuda/include -I/targets/x86_64-linux/include
I LDFLAGS:   -lcublas -lculibos -lcudart -lcublasLt -lpthread -ldl -lrt -L/usr/local/cuda/lib64 -L/opt/cuda/lib64 -L/targets/x86_64-linux/lib
I CC:       cc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
I CXX:      g++ (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0

make: Noth

In [ ]:
#!apt install nvidia-cuda-toolkit 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
nvidia-cuda-toolkit is already the newest version (10.1.243-3).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
!pip install diffusers
#esta version de transformers FUNCIONA, *NO TOCAR*
!pip install git+https://github.com/zphang/transformers.git@llama_push

!pip install langchain
!pip install -q datasets loralib sentencepiece 
!pip -q install bitsandbytes accelerate
!pip install safetensors
!pip install omegaconf
!pip install llama-index sentence_transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/zphang/transformers.git (to revision llama_push) to /tmp/pip-req-build-rd9kz9rp
  Running command git clone --filter=blob:none --quiet https://github.com/zphang/transformers.git /tmp/pip-req-build-rd9kz9rp
  Running command git checkout -b llama_push --track origin/llama_push
  Switched to a new branch 'llama_push'
  Branch 'llama_push' set up to track remote branch 'llama_push' from 'origin'.
  Resolved https://github.com/zphang/transformers.git to commit 3884da12ce327667d4df5101aef3533cc32be61f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.27.0.dev0-py3-none-any.whl size=6688506 sha256=d467b81d68f0fe6331bc162da3ae096

In [ ]:
#!pip install auto-gptq[llama] 
#!pip uninstall quant_cuda -y
#!pip install auto-gptq[llama]


In [ ]:
from transformers import GenerationConfig, pipeline, LlamaConfig, AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
import safetensors
import torch
#from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig


In [ ]:
import huggingface_hub
#huggingface_hub.snapshot_download("reeducator/vicuna-13b-cocktail", local_dir="./models")

#Seleccion y descarga del modelo:

- Elegir en los campos el repositorio de Huggingface y el archivo de "pesos" o "checkpoint" del modelo.

valores por defecto:
```
repo = "TheBloke/vicuna-13B-1.1-GPTQ-4bit-128g"
model_file = "vicuna-13B-1.1-GPTQ-4bit-128g.latest.safetensors"
```

In [ ]:
#@title Descarga del modelo
repo = "TheBloke/vicuna-13B-1.1-GPTQ-4bit-128g" #@param {type:"string"}
model_file = "vicuna-13B-1.1-GPTQ-4bit-128g.latest.safetensors" #@param {type:"string"}


from huggingface_hub import hf_hub_download


hf_hub_download(repo_id=repo, filename=model_file,local_dir="./model")
hf_hub_download(repo_id=repo, filename="quantize_config.json",local_dir="./model")
hf_hub_download(repo_id=repo, filename="config.json",local_dir="./model")
hf_hub_download(repo_id=repo, filename="generation_config.json",local_dir="./model")


model_path = "./model/" +  model_file

#Para la ejecucion por GPTQ:

In [ ]:
import sys
sys.path.append("GPTQ-for-LLaMa/")
import importlib
llama = importlib.import_module("llama_inference")
DEV = torch.device('cuda:0')
model = llama.load_quant(repo,model_path,4,128,0)
model.to(DEV)

Loading model ...


/usr/local/lib/python3.10/dist-packages/safetensors/torch.py:99: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(filename, framework="pt", device=device) as f:


Done.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (rotary_emb): LlamaRotaryEmbedding()
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): LlamaMLP(
          (act_fn): SiLUActivation()
          (down_proj): QuantLinear()
          (gate_proj): QuantLinear()
          (up_proj): QuantLinear()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_features=5120, out_features=32000, bias=False)
)

In [ ]:
#reeducator/vicuna-13b-cocktail
tokenizer = AutoTokenizer.from_pretrained(repo)

In [ ]:
from llama_index.prompts.prompts import SimpleInputPrompt

# This will wrap the default prompts that are internal to llama-index
# taken from https://huggingface.co/Writer/camel-5b-hf
query_wrapper_prompt = SimpleInputPrompt(
    "Below is an instruction that describes a task. "
    "Write a response that appropriately completes the request.\n\n"
    "### Instruction:\n{query_str}\n\n### Response:"
)


In [ ]:
import torch
from llama_index.llm_predictor import HuggingFaceLLMPredictor 

hf_predictor = HuggingFaceLLMPredictor(
    max_input_size=2048, 
    max_new_tokens=1024,
    temperature=0.25,
    do_sample=False,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name=repo,
    tokenizer = tokenizer,
    model_name=repo,
    model=model,
    device_map="auto",
    tokenizer_kwargs={"max_length": 2048},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}
)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install docx2txt pypdf
#!pip install InstructorEmbedding

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from llama_index import LangchainEmbedding
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

from llama_index import (
    GPTKeywordTableIndex,
    SimpleDirectoryReader,
    LLMPredictor,
    PromptHelper,
    ServiceContext,
    GPTVectorStoreIndex
)



In [ ]:
embed_model = LangchainEmbedding(HuggingFaceEmbeddings())

In [ ]:
chunk_size_limit = 512

service_context = ServiceContext.from_defaults(chunk_size_limit=chunk_size_limit, llm_predictor=hf_predictor,embed_model=embed_model)

In [ ]:
documents = SimpleDirectoryReader('/content/drive/MyDrive/test-data').load_data()



In [ ]:
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

In [ ]:

query_engine = index.as_query_engine(
    retriever_mode="embedding", 
    verbose=True, 
    service_context=service_context
)

In [ ]:
output=query_engine.query("How do I calculate a limit with complex numbers?")


In [ ]:
output.response

" \nTo calculate a limit with complex numbers, you can use the following steps:\n\n1. Simplify the expression by factoring the complex number into real and imaginary parts.\n2. Evaluate the limit of each part separately.\n3. Take the complex conjugate of the expression, and evaluate the limit of the complex conjugate.\n4. Combine the limits of the real and imaginary parts and the complex conjugate to find the limit of the original expression.\n5. If the limit exists, it will be a complex number. If the limit does not exist, the value of the expression at the point is infinity.\n\nFor example, let's try to find the limit of the function f(z) = z^2 + 2z as z approaches 2. We can simplify the expression as follows:\n\nf(z) = (2 + 2z) + 2z\n\nEvaluating the limit of each part separately, we get:\n\nlim(z->2) (2 + 2z) = 2\nlim(z->2) 2z = 0\n\nTaking the complex conjugate of the expression, we get:\n"

In [ ]:
output.source_nodes

[NodeWithScore(node=Node(text="= e; b) lím (az + b) = az0 + b (a #-O); z-zo z-+zo \ne) lím (z2 + e) = z5 + e; d) lím Re z = Re z0; e) \nz-zo z-+zo \nf) lím [x + i(2x + y)] = 1 + i (z = x + iy); z-+1-i g) lím (z2/z) = O. \nz-o \n2. Demostrar la afirmación [4] del Teorema 2, Sección 12a) usando el Teorema 1, \nSección 12, y propiedades de los límites de funciones reales de dos variables reales; \nb) directamente de la definición [2], Sección 11, de límite. \n3. Sea n un entero positivo, y sean P(z), Q(z) polinomios, con Q(z0) #-O. Usar el \nTeorema 2, Sección 12, y límites conocidos, para hallar ~ \n' 1 a) hm ---; ( z0 #-O); \nz-+zo Z iz3 -1 , P(z) b) lím · e) hm -· \nz-i z + i ' ,-,0 Q(z) \nSol. a) 1/~; b) O; e) P(z0)/Q(z0). \n4. Escribir L\\z = z -z0 y probar que \nlím f(z) = w0 si y sólo si lím /(z0 + L\\z) = w0. \nz-+zo .1.z-+O \n5. Demostrar que \nlím f(z)g(z) = O s1 lím /(z) = O \ny si existe un número positivo M tal que lg(z)I ~ M para todo zen algún entorno de \nZo. \n6. Probar",